# **Task 3: Object Detection Using YOLO**

In [1]:
# Import libraries for file handling and data splitting
import os, sys
import random
import shutil
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))
from logger import Logger
# Set instance of the Logger class
logger = Logger(log_file='../yolo_training.log')
sys.path.append('../yolov5')  # Adjust the path based on your directory structure

In [2]:
# Define the directory paths
photos_dir = '../data/photos'  # Directory containing all your images
train_dir = '../data/train/images'  # Directory for training images
val_dir = '../data/valid/images'  # Directory for validation images


try:
    # Create directories for train and validation sets if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    
    # Print the number of images available
    print(f"Total images available: {len(os.listdir(photos_dir))}")
    logger.info(f'{len(os.listdir(photos_dir))} total images available in {photos_dir}')
except Exception as e:
    logger.info(f'Error: {e}')

Total images available: 3821


In [3]:
def split_dataset(source_dir, train_dir, val_dir, split_ratio=0.8):
    # List all images in the source directory
    images = [f for f in os.listdir(source_dir) if f.endswith('.jpg')]
    random.shuffle(images)  # Shuffle the images for random selection

    # Calculate the split index
    split_index = int(len(images) * split_ratio)
    train_images = images[:split_index]
    val_images = images[split_index:]

    # Copy images to the respective directories
    for image in train_images:
        shutil.copy(os.path.join(source_dir, image), train_dir)
    for image in val_images:
        shutil.copy(os.path.join(source_dir, image), val_dir)
    
    print(f"Training images: {len(train_images)}")
    print(f"Validation images: {len(val_images)}")


if __name__=='__main__':
    
    try:
        # Call the function to split the dataset
        split_dataset(photos_dir, train_dir, val_dir)
        logger.info(f'Splited the dataset into train and validation sets.')
    except Exception as e:
        logger.info(f'Error: {e}')

Training images: 3056
Validation images: 765


In [4]:
# Clone the YOLOv5 repository (only run this if you don't have YOLOv5 cloned already)

# uncomment below line, and run the cell

!git clone https://github.com/ultralytics/yolov5.git

# Change the working directory to yolov5

# uncomment below line, and run the cell
%cd yolov5

# Install required packages for YOLOv5
# uncomment below line, and run the cell

%pip install -r requirements.txt

#' to automatically track and visualize YOLOv5 🚀 runs in Comet
# uncomment below line, and run the cell
%pip install comet_ml

Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.61 MiB | 770.00 KiB/s, done.
Resolving deltas: 100% (11690/11690), done.
/home/neba/Desktop/Data_Engineering/KAMI/Week-7/Ethiomed-Data-Warehouse/notebooks /yolov5


/home/neba/Desktop/Data_Engineering/KAMI/Week-7/Ethiomed-Data-Warehouse/.env/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="eDraTy7oHELDj6QbeQ2L")
project = rf.workspace("medical-products").project("medical-products")
version = project.version(1)
dataset = version.download("yolov5")

Note: you may need to restart the kernel to use updated packages.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to medical-products-1 in yolov5pytorch:: 100%|██████████| 392/392 [00:00<00:00, 6447.84it/s]


In [10]:
# Train the model using the pre-trained YOLOv5 model checkpoint (yolov5s.pt)
!python train.py --img 640 --batch 16 --epochs 50 --data ethio_medical_dataset.yml --weights yolov5s.pt
# Run this in your notebook cell
!python train.py --img 640 --batch 16 --epochs 50 --data ethio_medical_dataset.yml --weights yolov5s.pt --name fine_tuned_model
!python train.py --img 320 --batch 8 --epochs 50 --data ethio_medical_dataset.yml --weights yolov5n.pt --name fine_tuned_model

2024-10-25 14:01:57.169310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-25 14:01:57.202543: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-25 14:01:57.212507: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=ethio_medical_dataset.yml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_